In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from datasets import Dataset as HFDataset
import torch
import torch.nn.functional as F
import math
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandbpass")

In [3]:
import wandb
wandb.login(key=secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hexager (hexager-manipal). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
sentence = "By the way, we ball"
tokens = tokenizer.tokenize(sentence)
print(tokens)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

['by', 'the', 'way', ',', 'we', 'ball']


In [5]:
mnli_dataset = load_dataset("multi_nli")

README.md:   0%|          | 0.00/8.89k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/214M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/4.94M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/5.10M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [6]:
val_data = mnli_dataset["validation_matched"] # You can also try "validation_mismatched"
train_data = mnli_dataset["train"]

In [7]:
print(mnli_dataset['train'][0])

{'promptID': 31193, 'pairID': '31193n', 'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.', 'premise_binary_parse': '( ( Conceptually ( cream skimming ) ) ( ( has ( ( ( two ( basic dimensions ) ) - ) ( ( product and ) geography ) ) ) . ) )', 'premise_parse': '(ROOT (S (NP (JJ Conceptually) (NN cream) (NN skimming)) (VP (VBZ has) (NP (NP (CD two) (JJ basic) (NNS dimensions)) (: -) (NP (NN product) (CC and) (NN geography)))) (. .)))', 'hypothesis': 'Product and geography are what make cream skimming work. ', 'hypothesis_binary_parse': '( ( ( Product and ) geography ) ( ( are ( what ( make ( cream ( skimming work ) ) ) ) ) . ) )', 'hypothesis_parse': '(ROOT (S (NP (NN Product) (CC and) (NN geography)) (VP (VBP are) (SBAR (WHNP (WP what)) (S (VP (VBP make) (NP (NP (NN cream)) (VP (VBG skimming) (NP (NN work)))))))) (. .)))', 'genre': 'government', 'label': 1}


In [8]:
max_seq_length = 128

In [9]:
def tokenize_mnli(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], max_length=max_seq_length, padding='max_length', truncation=True)

tokenized_train_data = mnli_dataset["train"].map(tokenize_mnli, batched=True, remove_columns=['premise', 'hypothesis'])
tokenized_val_data = mnli_dataset["validation_matched"].map(tokenize_mnli, batched=True, remove_columns=['premise', 'hypothesis'])

class MNLIDatasetOptimized(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'labels': torch.tensor(item['label'])
        }

train_dataset = MNLIDatasetOptimized(tokenized_train_data)
val_dataset = MNLIDatasetOptimized(tokenized_val_data)

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

In [10]:
batch_size_finetune = 32 # You can adjust this
train_dataloader = DataLoader(train_dataset, batch_size=batch_size_finetune, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size_finetune)

In [11]:
sample = train_dataset[0]
print(sample['input_ids'].shape)
print(sample['attention_mask'].shape)
print(sample['labels'])

torch.Size([128])
torch.Size([128])
tensor(1)


In [12]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, dropout_rate):
        super().__init__()
        self.num_heads = num_attention_heads
        self.head_dim = hidden_size // num_attention_heads
        assert self.head_dim * self.num_heads == hidden_size

        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)

        self.dropout = nn.Dropout(dropout_rate)
        self.output = nn.Linear(hidden_size, hidden_size)
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        seq_len_q, seq_len_k, seq_len_v = query.size(1), key.size(1), value.size(1)
        query = self.query(query)
        key = self.key(key)
        value = self.value(value)
        query = query.view(batch_size, seq_len_q, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        key = key.view(batch_size, seq_len_k, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        value = value.view(batch_size, seq_len_v, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        attention_scores = torch.matmul(query, key.transpose(-2,-1))
        attention_scores = attention_scores/(self.head_dim**0.5)
        mask = mask.unsqueeze(1).unsqueeze(2)
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask==0, float('-inf'))
        attention_weights = F.softmax(attention_scores,dim=-1)
        scaled_attention = torch.matmul(attention_weights, value)
        scaled_attention = scaled_attention.permute(0, 2, 1, 3).contiguous().view(batch_size, seq_len_q, self.num_heads * self.head_dim)
        output = self.output(scaled_attention)
        return output

In [13]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, hidden_size, intermediate_size, dropout_rate):
        super().__init__()
        self.dense1 = nn.Linear(hidden_size, intermediate_size)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(intermediate_size, hidden_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = self.dense2(x)
        return x

In [14]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, hidden_size, num_attention_heads, intermediate_size, dropout_rate):
        super().__init__()
        self.self_attention = MultiHeadSelfAttention(hidden_size, num_attention_heads, dropout_rate)
        self.feed_forward = FeedForwardNetwork(hidden_size, intermediate_size, dropout_rate)
        self.norm1 = nn.LayerNorm(hidden_size)
        # Removed the second LayerNorm (norm2)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, mask):
        # Self-attention with residual and normalization
        attention_output = self.self_attention(x, x, x, mask)
        normed_output1 = self.norm1(attention_output + x)
        
        # Feed-forward block with residual, no additional normalization here
        ff_output = self.feed_forward(normed_output1)
        final_output = ff_output + normed_output1
        return final_output


In [15]:
class FactorizedEmbedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, type_vocab_size=2):
        super().__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.token_type_embeddings = nn.Embedding(type_vocab_size, embedding_dim)
        self.projection = nn.Linear(embedding_dim, hidden_size)

    def forward(self, input_ids, token_type_ids=None):
        factor_embeds = self.word_embeddings(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        token_type_embeds = self.token_type_embeddings(token_type_ids)
        # Sum the embeddings
        embeds = factor_embeds + token_type_embeds        
        project_embeds = self.projection(embeds)
        return project_embeds

In [16]:
class PositionalEncoding(nn.Module):
    def __init__(self, hidden_size, max_seq_length, dropout_rate):
        super().__init__()
        self.dropout = nn.Dropout(dropout_rate)

        position = torch.arange(0, max_seq_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_size, 2) * -(math.log(10000.0) / hidden_size))
        pe = torch.zeros(max_seq_length, 1, hidden_size)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe) # Store as a buffer (not a learnable parameter)

    def forward(self, x):
        seq_length = x.size(1)
        pe = self.pe[:seq_length].squeeze(1)
        x = x + pe.unsqueeze(0)
        x = self.dropout(x)
        return x

In [17]:
class AtomBERT(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, 
                 num_attention_heads, intermediate_size, num_classes, max_seq_length, 
                 dropout_rate, type_vocab_size=2):
        super().__init__()
        self.embedding = FactorizedEmbedding(vocab_size, embedding_dim, hidden_size, type_vocab_size)
        self.positional_encoding = PositionalEncoding(hidden_size, max_seq_length, dropout_rate)
        # Shared encoder layer across all iterations
        self.encoder_layer = TransformerEncoderLayer(hidden_size, num_attention_heads, intermediate_size, dropout_rate)
        # Create separate layer norms for each iteration
        self.layer_norms = nn.ModuleList([nn.LayerNorm(hidden_size) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout_rate)
        self.num_layers = num_layers
        self.hidden_size = hidden_size

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        embeddings = self.embedding(input_ids, token_type_ids)
        embeddings = self.positional_encoding(embeddings)

        encoder_output = embeddings
        for ln in self.layer_norms:
            # Pass through the shared encoder layer
            output = self.encoder_layer(encoder_output, attention_mask)
            # Residual connection plus per-iteration layer normalization
            encoder_output = ln(encoder_output + output)
        pooled_output = encoder_output[:, 0, :]  # Use [CLS] token
        pooled_output = self.dropout(pooled_output)
        return encoder_output, pooled_output


In [18]:
class AtomBERTForPretraining(nn.Module): # Renamed for clarity
    def __init__(self, config):
        super().__init__()
        self.config = config # Store the config object
        self.bert = AtomBERT(
            vocab_size=config.vocab_size,
            embedding_dim=config.embedding_dim,
            hidden_size=config.hidden_size,
            num_layers=config.num_layers,
            num_attention_heads=config.num_attention_heads,
            intermediate_size=config.intermediate_size,
            num_classes=2, # For SOP
            max_seq_length=config.max_seq_length,
            dropout_rate=config.dropout_rate,
            type_vocab_size=config.type_vocab_size if hasattr(config, 'type_vocab_size') else 2
        )
        self.mlm_head = nn.Sequential(
        nn.Linear(config.hidden_size, config.hidden_size),
        nn.LayerNorm(config.hidden_size),
        nn.Linear(config.hidden_size, config.vocab_size)
        )
        self.sop_head = nn.Linear(config.hidden_size, 2)



    def forward(self, input_ids, attention_mask, masked_labels=None, sentence_order_labels=None, token_type_ids=None):
        outputs = self.bert(input_ids, attention_mask, token_type_ids) # Shape: (batch_size, seq_len, hidden_size)
        encoder_output = outputs[0]
        pooled_output = outputs[1]
        # MLM Prediction
        prediction_logits_mlm = self.mlm_head(encoder_output) # Shape: (batch_size, seq_len, vocab_size)

        # SOP Prediction (using the pooled output - we might need to adjust this)
        #pooled_output = outputs[:, 0, :] # Taking the [CLS] token representation
        prediction_logits_sop = self.sop_head(pooled_output) # Shape: (batch_size, 2)

        total_loss = None
        if masked_labels is not None and sentence_order_labels is not None:
            loss_fct_mlm = nn.CrossEntropyLoss()
            masked_loss = loss_fct_mlm(prediction_logits_mlm.view(-1, self.config.vocab_size), masked_labels.view(-1))

            loss_fct_sop = nn.CrossEntropyLoss()
            sop_loss = loss_fct_sop(prediction_logits_sop.view(-1, 2), sentence_order_labels.view(-1))

            total_loss = masked_loss + sop_loss # You might want to weigh these differently

        elif masked_labels is not None:
            loss_fct_mlm = nn.CrossEntropyLoss()
            total_loss = loss_fct_mlm(prediction_logits_mlm.view(-1, self.config.vocab_size), masked_labels.view(-1))

        elif sentence_order_labels is not None:
            loss_fct_sop = nn.CrossEntropyLoss()
            total_loss = loss_fct_sop(prediction_logits_sop.view(-1, 2), sentence_order_labels.view(-1))

        return total_loss, prediction_logits_mlm, prediction_logits_sop

In [19]:
class AtomBERTConfig:
    def __init__(
        self,
        vocab_size,
        embedding_dim=128,
        hidden_size=768,
        num_layers=4,
        num_attention_heads=4,
        intermediate_size=1500,
        num_classes=2, # For SOP
        max_seq_length=128,
        dropout_rate=0.1,
        type_vocab_size = 2
    ):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_attention_heads = num_attention_heads
        self.intermediate_size = intermediate_size
        self.num_classes = num_classes
        self.max_seq_length = max_seq_length
        self.dropout_rate = dropout_rate
        self.type_vocab_size = type_vocab_size

In [20]:
wiki_text_dataset = load_dataset("wikitext", "wikitext-103-raw-v1")
print(wiki_text_dataset)

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [21]:
vocab_size = tokenizer.vocab_size
learning_rate = 5e-5

# Move the model to the device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Recall or re-initialize your tokenizer if needed
vocab_size = tokenizer.vocab_size

# Create the configuration
config = AtomBERTConfig(vocab_size=vocab_size) # You can customize other parameters here if needed

# Instantiate the AtomBERTForPretraining model
modelpretrain = AtomBERTForPretraining(config)

print(f"Model instantiated with {config}")

Model instantiated with <__main__.AtomBERTConfig object at 0x78be298f4e80>


In [22]:
def count_parameters(model):
    total_params = 0
    trainable_params = 0
    non_trainable_params = 0

    for name, param in model.named_parameters():
        num_params = param.numel()
        # if requires_grad is True, then it is a trainable parameter
        if param.requires_grad:
            trainable_params += num_params
        else:
            non_trainable_params += num_params
        total_params += num_params

    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}")
    print(f"Non-trainable Parameters: {non_trainable_params:,}")
print("Basic Model: \n")
count_parameters(modelpretrain.bert)
print("Total Pretraining Model: \n")
count_parameters(modelpretrain)


Basic Model: 

Total Parameters: 8,682,460
Trainable Parameters: 8,682,460
Non-trainable Parameters: 0
Total Pretraining Model: 

Total Parameters: 32,747,544
Trainable Parameters: 32,747,544
Non-trainable Parameters: 0


In [23]:
# Start a new wandb run to track this script.
import wandb
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="hexager-manipal",
    # Set the wandb project where this run will be logged.
    project="MRM-transform_MOD",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "AlBERT-1M",
        "dataset": "WikiText-103",
        "epochs": 4,
    },
)

wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250330_062528-lh5glq4m
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sleek-haze-2
wandb: ⭐️ View project at https://wandb.ai/hexager-manipal/MRM-transform_MOD
wandb: 🚀 View run at https://wandb.ai/hexager-manipal/MRM-transform_MOD/runs/lh5glq4m


In [24]:
# Define parameters (ensure max_seq_length and batch_size are defined)
max_seq_length = 128  # Example value, adjust as needed
batch_size = 64      # Example value, adjust as needed
num_epochs = 2
learning_rate = 1e-4
weight_decay = 0.01
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Load tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Load dataset
wiki_text_dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=max_seq_length)
tokenized_train_dataset = wiki_text_dataset["train"].map(tokenize_function, batched=True)

# Prepare MLM data
def prepare_mlm_data(examples):
    input_ids = torch.tensor(examples["input_ids"])
    batch_size_local = len(input_ids)
    sequence_length = len(input_ids[0])
    mask_prob = 0.15
    mask_token_id = tokenizer.mask_token_id
    pad_token_id = tokenizer.pad_token_id
    cls_token_id = tokenizer.cls_token_id
    sep_token_id = tokenizer.sep_token_id

    rand = torch.rand(batch_size_local, sequence_length)
    mask = (rand < mask_prob) & (input_ids != cls_token_id) & (input_ids != sep_token_id) & (input_ids != pad_token_id)

    labels = input_ids.clone()
    labels[~mask] = -100
    inputs = input_ids.clone()
    inputs[mask] = mask_token_id

    return {"input_ids": inputs, "labels": labels}
mlm_tokenized_train_dataset = tokenized_train_dataset.map(
    prepare_mlm_data,
    batched=True,
    remove_columns=["text", "token_type_ids"]
)

# Prepare SOP data
import re
from datasets import Dataset as HFDataset

def split_into_sentences_robust(text):
    sentence_pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s+'
    sentences = re.split(sentence_pattern, text)
    return [s.strip() for s in sentences if s.strip()]

sop_texts_limited = []
sop_labels_limited = []
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
max_pairs_per_document = 5

for example in wiki_text_dataset["train"]:
    text = example["text"]
    sentences = split_into_sentences_robust(text)
    pairs_count = 0
    for i in range(len(sentences) - 1):
        if pairs_count >= max_pairs_per_document:
            break
        sentence_a = sentences[i]
        sentence_b = sentences[i + 1]
        sop_texts_limited.append(cls_token + " " + sentence_a + " " + sep_token + " " + sentence_b)
        sop_labels_limited.append(0)
        pairs_count += 1
        if pairs_count >= max_pairs_per_document:
            break
        sop_texts_limited.append(cls_token + " " + sentence_b + " " + sep_token + " " + sentence_a)
        sop_labels_limited.append(1)
        pairs_count += 1

sop_dataset = HFDataset.from_dict({"text": sop_texts_limited, "sentence_order_labels": sop_labels_limited})

def tokenize_sop_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length', max_length=max_seq_length)
tokenized_sop_dataset = sop_dataset.map(tokenize_sop_function, batched=True)

# Set format to PyTorch tensors
mlm_tokenized_train_dataset.set_format("torch")
tokenized_sop_dataset.set_format("torch")

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True) # Adjust num_workers
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4, pin_memory=True) # Adjust num_workers
mlm_dataloader = DataLoader(
    mlm_tokenized_train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4, # Adjust num_workers
    pin_memory=True
)
sop_dataloader = DataLoader(
    tokenized_sop_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4, # Adjust num_workers
    pin_memory=True
)

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Map:   0%|          | 0/3189245 [00:00<?, ? examples/s]

In [25]:
modelpretrain.to(device)
from torch.optim import AdamW
optimizer = AdamW(modelpretrain.parameters(), lr=learning_rate, weight_decay=weight_decay)
wandb.watch(modelpretrain)

In [26]:
counter = 0
for epoch in range(num_epochs):
    modelpretrain.train()
    total_mlm_loss = 0.0
    total_sop_loss = 0.0

    # Use zip to iterate concurrently over both dataloaders
    total_steps = min(len(mlm_dataloader), len(sop_dataloader))
    progress_bar = tqdm(zip(mlm_dataloader, sop_dataloader), total=total_steps, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch_mlm, batch_sop in progress_bar:
        counter += 1
        # Transfer batches to the device
        mlm_inputs = {k: v.to(device) for k, v in batch_mlm.items()}
        sop_inputs = {
                    k: v.to(device) if isinstance(v, torch.Tensor) else v
                    for k, v in batch_sop.items()
                    }
        optimizer.zero_grad()

        # MLM forward pass
        returned_loss, _, _ = modelpretrain(
            input_ids=mlm_inputs['input_ids'],
            attention_mask=mlm_inputs['attention_mask'],
            masked_labels=mlm_inputs['labels']
        )
        mlm_loss = returned_loss

        # SOP forward pass
        outputs_sop = modelpretrain(
            input_ids=sop_inputs['input_ids'],
            attention_mask=sop_inputs['attention_mask'],
            token_type_ids=sop_inputs.get('token_type_ids', None),
            sentence_order_labels=sop_inputs['sentence_order_labels']
        )
        sop_loss = outputs_sop[0]

        # Combine losses and update
        total_loss = mlm_loss + sop_loss
        total_loss.backward()
        optimizer.step()

        total_mlm_loss += mlm_loss.item()
        total_sop_loss += sop_loss.item()

        # Log every step (or throttle this if needed)
        if(counter%15==0):
            wandb.log({
                "mlm_loss": mlm_loss.item(), 
                "sop_loss": sop_loss.item(), 
                "total_loss": total_loss.item()
            })
        progress_bar.set_postfix({
            "mlm_loss": f"{mlm_loss.item():.4f}", 
            "sop_loss": f"{sop_loss.item():.4f}"
        })

    avg_mlm_loss = total_mlm_loss / total_steps if total_steps > 0 else 0
    avg_sop_loss = total_sop_loss / total_steps if total_steps > 0 else 0

    # Logging epoch metrics
    wandb.log({
        "avg_mlm_loss": avg_mlm_loss, 
        "avg_sop_loss": avg_sop_loss, 
        "epoch": epoch + 1,
        "Total Unsupervised Loss": total_loss.item()  # Note: last step's loss
    })

    # Save checkpoint
    save_path = '/kaggle/working/pretrained_model.pth'
    torch.save(modelpretrain.state_dict(), save_path)
    wandb.save(save_path)
    print(f"Epoch {epoch+1}/{num_epochs} finished, Average MLM Loss: {avg_mlm_loss:.4f}, Average SOP Loss: {avg_sop_loss:.4f}")


Epoch 1/2:   0%|          | 0/28147 [00:00<?, ?it/s]

wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Epoch 1/2 finished, Average MLM Loss: 5.3874, Average SOP Loss: 0.6316


Epoch 2/2:   0%|          | 0/28147 [00:00<?, ?it/s]

Epoch 2/2 finished, Average MLM Loss: 4.4161, Average SOP Loss: 0.5797


In [27]:
save_path = '/kaggle/working/pretrained_model.pth'  # Replace with your desired path
torch.save(modelpretrain.state_dict(), save_path)
wandb.save(save_path)


['/kaggle/working/wandb/run-20250330_062528-lh5glq4m/files/working/pretrained_model.pth']

In [28]:
class AtomBERTForSequenceClassification(nn.Module):
    def __init__(self, pretrained_config, num_classes):
        super().__init__()
        self.bert = AtomBERT(
            vocab_size=pretrained_config.vocab_size,
            embedding_dim=pretrained_config.embedding_dim,
            hidden_size=pretrained_config.hidden_size,
            num_layers=pretrained_config.num_layers,
            num_attention_heads=pretrained_config.num_attention_heads,
            intermediate_size=pretrained_config.intermediate_size,
            num_classes=num_classes, # For MNLI this will be 3
            max_seq_length=pretrained_config.max_seq_length,
            dropout_rate=pretrained_config.dropout_rate,
            type_vocab_size=pretrained_config.type_vocab_size if hasattr(pretrained_config, 'type_vocab_size') else 2
        )
        self.dropout = nn.Dropout(pretrained_config.dropout_rate)
        self.classifier = nn.Linear(pretrained_config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask)
        pooled_output = outputs[1] # We typically use the pooled output for classification
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [29]:


# Define the path to your saved pretrained weights file
pretrained_weights_path = '/kaggle/working/pretrained_model.pth' # Replace with the actual path to your saved file

# Load the configuration of your pretrained model
# Make sure these values match what you used for pretraining
pretrained_config = AtomBERTConfig(
    vocab_size=tokenizer.vocab_size,
    embedding_dim=128,
    hidden_size=768,
    num_layers=4,
    num_attention_heads=4,
    intermediate_size=1500,
    max_seq_length=128,
    dropout_rate=0.1,
    type_vocab_size=2
)

# Instantiate the model for sequence classification (for MNLI with 3 classes)
num_classes_mnli = 3
model_for_mnli = AtomBERTForSequenceClassification(pretrained_config, num_classes_mnli)

# Load the state dictionary of the pretrained model
pretrained_state_dict = torch.load(pretrained_weights_path, map_location=torch.device(device)) # Ensure device consistency

# Create a new dictionary to store only the 'bert' weights
pretrained_bert_state_dict = {}
for name, param in pretrained_state_dict.items():
    if name.startswith('bert.'):
        pretrained_bert_state_dict[name[len('bert.'):]] = param

# Load the pretrained weights into the 'bert' part of the sequence classification model
model_for_mnli.bert.load_state_dict(pretrained_bert_state_dict)

print("Pretrained weights loaded successfully into the sequence classification model.")

# Move the fine-tuning model to the device
model_for_mnli.to(device)

Pretrained weights loaded successfully into the sequence classification model.


<ipython-input-29-6f785b1d3dda>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state_dict = torch.load(pretrained_weights_path, map_location=torch.device(devic

AtomBERTForSequenceClassification(
  (bert): AtomBERT(
    (embedding): FactorizedEmbedding(
      (word_embeddings): Embedding(30522, 128)
      (token_type_embeddings): Embedding(2, 128)
      (projection): Linear(in_features=128, out_features=768, bias=True)
    )
    (positional_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder_layer): TransformerEncoderLayer(
      (self_attention): MultiHeadSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (output): Linear(in_features=768, out_features=768, bias=True)
      )
      (feed_forward): FeedForwardNetwork(
        (dense1): Linear(in_features=768, out_features=1500, bias=True)
        (relu): ReLU()
        (dense2): Linear(in_features=1500, out_features=768, bias

In [30]:
model_finetune = AtomBERTForSequenceClassification(pretrained_config, 3)
model_finetune.to(device)
# 5. Load the pretrained weights into the new model
# Load the state dictionary of the pretrained model
pretrained_state_dict = torch.load(pretrained_weights_path, map_location=torch.device(device)) # Ensure device consistency

# Create a new dictionary to store only the 'bert' weights
pretrained_bert_state_dict = {}
for name, param in pretrained_state_dict.items():
    if name.startswith('bert.'):
        pretrained_bert_state_dict[name[len('bert.'):]] = param

# Load the pretrained weights into the 'bert' part of the sequence classification model
model_finetune.bert.load_state_dict(pretrained_bert_state_dict)
learning_rate_finetune = 2e-5
optimizer = AdamW(model_finetune.parameters(), lr=learning_rate_finetune)
model_finetune.to(device)
# 9. Define the loss function separately
criterion = nn.CrossEntropyLoss()
# 10. Define your training loop (modified for custom model)
num_epochs_finetune = 3
for epoch in range(num_epochs_finetune):
    model_finetune.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs_finetune}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model_finetune(input_ids, attention_mask=attention_mask) # Forward pass without labels
        logits = outputs # Assuming your model's forward returns the logits directly
        loss = criterion(logits, labels) # Calculate the loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Training Loss: {total_loss/len(train_dataloader)}")

    # Evaluation loop (modified for custom model)
    model_finetune.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model_finetune(input_ids, attention_mask=attention_mask) # Forward pass without labels
            logits = outputs
            _, predicted = torch.max(logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1} Validation Accuracy: {accuracy:.2f}%")

print("Fine-tuning finished!")

<ipython-input-30-deff463ce58f>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_state_dict = torch.load(pretrained_weights_path, map_location=torch.device(device

Epoch 1/3:   0%|          | 0/6136 [00:00<?, ?it/s]

Epoch 1 Training Loss: 0.9419112735011431


Evaluating:   0%|          | 0/154 [00:00<?, ?it/s]

Epoch 1 Validation Accuracy: 61.20%


...

Epoch 2 Validation Accuracy: 64.57%


Epoch 3/3:   0%|          | 0/6136 [00:00<?, ?it/s]

Epoch 3 Training Loss: 0.7615210018867464


Evaluating:   0%|          | 0/154 [00:00<?, ?it/s]

Epoch 3 Validation Accuracy: 66.32%
Fine-tuning finished!
